# <span style="color:yellow">ΚΑΘΟΡΙΣΜΟΣ ΤΟΥ DIRECTORY ΟΠΟΥ ΒΡΙΣΚΕΤΑΙ ΤΟ ΑΡΧΕΙΟ

In [ ]:
pwd #Καθορίζει το path file του αρχείου που εκτελείται

# <span style="color:yellow">ΕΙΣΑΓΩΓΗ ΤΩΝ ΑΠΑΡΑΙΤΡΗΤΩΝ ΒΙΒΛΙΟΘΗΚΩΝ ΓΙΑ ΤΗΝ ΕΚΤΕΛΕΣΗ ΤΟΥ ΚΩΔΙΚΑ


In [ ]:
#Βιβλιοθήκες για την εισαγωγή και επεξεργασία των δεδομένων
import warnings
import numpy as np
import pandas as pd

#Βιβλιοθήκες για την εκτύπωση των διαγραμμάτων
import seaborn as sns
import matplotlib.cm as cm
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Βιβλιοθήκες για εκπαίδευση μοντέλων μηχανικής μάθησης
from sklearn.svm import SVR
from arch import arch_model
import statsmodels.api as sm
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# <span style="color:yellow">ΚΑΘΟΡΙΣΜΟΣ ΑΚΡΙΒΕΙΑΣ ΤΟΥ ΜΟΝΤΕΛΟΥ

In [ ]:
#Καθορισμός ακριβείας του μοντέλου
pd.set_option("display.precision", 2)

# <span style="color:yellow">ΕΙΣΑΓΩΓΗ ΚΑΙ ΕΠΕΞΕΡΓΑΣΙΑ ΤΟΥ ΑΡΧΕΙΟΥ ΠΟΥ ΕΙΝΑΙ ΑΠΑΡΑΙΤΗΤΟ ΓΙΑ ΤΗΝ ΕΚΤΕΛΕΣΗ ΤΟΥ ΑΛΓΟΡΙΘΜΟΥ

In [ ]:
#Αναγνώριση του αρχείου και εισαγωγή του στο πρόγραμμα
df = pd.read_excel('../FINAL_DATA/Ireland_Values.xlsx')

#Δημιουργία DataFrame
df.head()

#Δίνει την εκτύπωση των πέντε πρώτων στηλών των δεδομένων
print(df.head())

# <span style="color:yellow">ΣΥΝΟΠΤΙΚΗ ΠΑΡΟΥΣΙΑΣΗ ΤΩΝ ΔΕΔΟΜΕΝΩΝ

In [ ]:
#Επιστρέφει το σύνολο των γραμμών του αρχείου στο οποίο βρίσκονται τα δεδομένα
len(df)

In [ ]:
#Επιστρέφει τα ονόματα των στηλών του αρχείου στο οποίο βρίσκονται τα δεδομένα
df.columns

In [ ]:
# Επιστρέφει το σύνολο όλων των τιμών στο DataFrame
total_values = df.size

#Aυτή η γραμμή κώδικα εκτυπώνει το συνολικό πλήθος των δεδομένων που περιέχονται στο `DataFrame`
print(f"Σύνολο δεδομένων στο DataFrame: {total_values}")

In [ ]:
#Δίνει τον τύπο των δεδομένων που περιέχονται στις στήλες του αρχείου. To dataset αποτελείται από 11 στήλες από τις οποίες οι δέκα περιέχουν αριθμητικά δεδομένα και η μια ημερομηνία
df.dtypes

In [ ]:
# Εκτυπώνει το όνομα των στηλών και τον τύπο των δεδομένων που περιέχουν
df.info()

In [ ]:
#Εκτυπώνει το είδος της κάθε μεταβλητής του dataset
print(df.dtypes)

In [ ]:
#Δίνει τον πίνακα συσχέτισης των μεταβλητών του dataset
df.corr()

# <span style="color:yellow">ΠΡΟΕΤΟΙΜΑΣΙΑ ΤΩΝ ΔΕΟΔΜΕΝΩΝ

In [ ]:
# Έλεγχος για ελλείπουσες τιμές (NaN)
missing_values = df.isnull().sum()

# Εμφάνιση των αποτελεσμάτων
print("Ελλείπουσες τιμές ανά στήλη:")
print(missing_values)

In [ ]:
# Υπολογισμός Z-Scores για αριθμητικές στήλες
z_scores = df.select_dtypes(include=[np.number]).apply(zscore)

# Καθορισμός ακραιών τιμών (π.χ., όριο Z-Score > |3|)
threshold = 3
outliers = (z_scores.abs() > threshold)

# Εμφάνιση ακραίων τιμών
print("Ακραιες τιμές ανά στήλη:")
for column in outliers.columns:
    print(f"\nΣτήλη: {column}")
    print(df.loc[outliers[column], column])

In [ ]:
# Φιλτράρισμα δεδομένων για απομάκρυνση γραμμών με ακραίες τιμές
df_without_outliers = df[~(z_scores.abs() > threshold).any(axis=1)]

# Εμφάνιση του περιεχομένου του DataFrame μετά την αφαίρεση των ακραίων τιμών
print("\nDataFrame μετά την αφαίρεση ακραίων τιμών:")
print(df_without_outliers)

In [ ]:
# Μετατροπή της στήλης 'Date' σε datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d',
                            errors='coerce')  # Προσαρμόστε το format ανάλογα με τη μορφή της ημερομηνίας σας

# Έλεγχος αν η μετατροπή ήταν επιτυχής
print(df['Date'].head())

# Προαιρετικά: Ρύθμιση της στήλης 'Date' ως index
df.set_index('Date', inplace=True)

# Έλεγχος του DataFrame
print(df.head())

In [ ]:
# Χρησιμοποιείται το index του DataFrame, το οποίο είναι τύπου DatetimeIndex.
df.index

# Το DataFrame ταξινομείται με βάση το index του σε αύξουσα σειρά.
df = df.sort_index()

# Εκτυπώνονται οι πρώτες γραμμές του ταξινομημένου DataFrame για επισκόπηση.
print(df.head())

# <span style="color:yellow">Δημιουργία Χρονικών Υστερήσεων (Lagged Variables)

In [ ]:
# Ορισμός των χρονικών υστερήσεων (lags) που θέλουμε
lags = [7, 14, 21]  #

# Επιλογή των αριθμητικών στηλών στο DataFrame
columns_to_lag = df.select_dtypes(include=[np.number]).columns

# Δημιουργία νέου DataFrame για τις lagged μεταβλητές
lagged_data = pd.DataFrame(index=df.index)

# Δημιουργία lagged μεταβλητών για κάθε επιλεγμένη στήλη
for col in columns_to_lag:
    for lag in lags:
        lagged_data[f"{col}_lag_{lag}"] = df[col].shift(lag)

# Ενοποίηση του αρχικού DataFrame με τις lagged μεταβλητές
df = pd.concat([df, lagged_data], axis=1)

# Αφαίρεση των γραμμών που περιέχουν NaN λόγω των χρονικών υστερήσεων
df.dropna(inplace=True)

# Επισκόπηση του τελικού DataFrame
print("\nDataFrame με υστερήσεις 7, 14, 21 ημερών:")
print(df.head())

# <span style="color:yellow">Καθορισμός Μεταβλητών Μοντέλου SVR

In [ ]:
# Καθορισμός της εξαρτημένης μεταβλητής (μεταβλητότητα)
dependent_var = "7- Day Standard Deviation of Daily Returns"

In [ ]:
# Δημιουργία του X (independent) και y (dependent)
X = df.drop(columns=[dependent_var])
y = df[dependent_var]

# <span style="color:yellow">Διαχωρισμός σε train (80%) - test (20%) split

In [ ]:
# Το 80% των δεδομένων χρησιμοποιείται για εκπαίδευση
train_size = int(len(df) * 0.8)

# Διαχωρισμός χαρακτηριστικών σε train και test set
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]

# Διαχωρισμός target μεταβλητής σε train και test set
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]


# Επισκόπηση μεγεθών των training και test set.
print(f"Μέγεθος X_train: {X_train.shape}")
print(f"Μέγεθος X_test: {X_test.shape}")
print(f"Μέγεθος y_train: {y_train.shape}")
print(f"Μέγεθος y_test: {y_test.shape}")


# Εκτύπωση τιμών στο Training και Test Set
print("\nΠρώτες γραμμές του X_train:")
print(X_train.head())

print("\nΠρώτες γραμμές του X_test:")
print(X_test.head())

# <span style="color:yellow">Κανονικοποίηση Δεδομένων

In [ ]:
# Υπολογισμός μέσου όρου και τυπικής απόκλισης
X_train_mean = X_train.mean()
X_train_std = X_train.std()

# Κανονικοποίηση δεδομένων
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

# Εκτύπωση αποτελεσμάτων
print("Μέσος όρος των χαρακτηριστικών του X_train:")
print(X_train_mean)

print("\nΤυπική απόκλιση των χαρακτηριστικών του X_train:")
print(X_train_std)

print("\nΠρώτες 5 γραμμές του κανονικοποιημένου X_train:")
print(X_train.head())

print("\nΠρώτες 5 γραμμές του κανονικοποιημένου X_test:")
print(X_test.head())

# <span style="color:yellow">Support Vector Regression Model (SVR)

In [ ]:
# Δημιουργία του μοντέλου SVR
model= svr = SVR(kernel="linear", C=1.0)  # Χρησιμοποιείται γραμμικός πυρήνας

In [ ]:
# Εκπαίδευση του μοντέλου SVR
svr.fit(X_train, y_train)

In [ ]:
# Πρόβλεψη στο Training Set
y_train_pred = svr.predict(X_train)

# Εκτύπωση των πρώτων 10 αποτελεσμάτων της πρόβλεψης
print("Προβλέψεις για το X_train:")
print(y_train_pred[:10])

In [ ]:
# Πρόβλεψη αποτελεσμάτων για το X_test
y_pred = svr.predict(X_test)

# Εκτύπωση των πρώτων 10 αποτελεσμάτων της πρόβλεψης
print("Προβλέψεις για το X_test:")
print(y_pred[:10])

# <span style="color:yellow">Support Vector Regression Model (y_train)

In [ ]:
# Υπολογισμός μετρικών αξιολόγησης
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

# Εκτύπωση αποτελεσμάτων αξιολόγησης
print("Αξιολόγηση στο Training Set:")
print(f"Mean Squared Error (MSE): {mse_train:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_train:.4f}")
print(f"R^2 Score: {r2_train:.4f}")

# Δημιουργία DataFrame για τις μετρικές
metrics_df = pd.DataFrame({
    "Metric": ["Mean Squared Error (MSE)", "Root Mean Squared Error (RMSE)", "R² Score"],
    "Value": [mse_train, rmse_train, r2_train]
})

# Αποθήκευση των μετρικών σε αρχείο Excel
metrics_df.to_excel("training_metrics-train.xlsx", index=False)

# <span style="color:yellow">Support Vector Regression Model (y_test)

In [ ]:
# Υπολογισμός των μετρικών αξιολόγησης του μοντέλου SVR
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Εκτύπωση των αποτελεσμάτων
print("Αξιολόγηση στο Test Set:")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
print(f"R² Score: {r2:.3f}")

# Δημιουργία DataFrame για τις μετρικές
metrics_df = pd.DataFrame({
    "Metric": ["Mean Squared Error (MSE)", "Root Mean Squared Error (RMSE)", "R² Score"],
    "Value": [mse, rmse, r2]
})

# Αποθήκευση των μετρικών σε αρχείο Excel
metrics_df.to_excel("training_metrics-test.xlsx", index=False)

# <span style="color:yellow">Πρόβλεψη για τις επόμενες 7 ημέρες

In [ ]:
# Εισαγωγή της τελευταίας γραμμής από το dataset
latest_features = X_test.iloc[-1].copy()

# Καθαρισμός της γραμμής από NaN τιμές (αντικατάσταση με 0)
latest_features.fillna(0, inplace=True)

print("Καθαρισμένη Τελευταία Γραμμή από το X_test (latest_features):")
print(latest_features)

In [ ]:
# Πίνακας για νέα δεδομένα, δημιουργούνται 7 ημερομηνίες
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=7, freq="D")

# Εκτύπωση για τον έλεγχο των μελλοντικών ημερομηνιών
print("Νέες ημερομηνίες για τις προβλέψεις:")
print(future_dates)

# Αποθήκευση προβλέψεων
future_predictions = []

# Έλεγχος NaN τιμών και αντικατάστασή τους αν χρειάζεται
if latest_features.isnull().any():
    print("\nΕντοπίστηκαν NaN τιμές στην τελευταία γραμμή! Αντικαθίστανται με τον μέσο όρο των χαρακτηριστικών.")
    latest_features = latest_features.fillna(X_train.mean())

In [ ]:
# Απενεργοποιεί όλα τα warnings
warnings.filterwarnings("ignore")

# Ενσωμάτωση μοντέλου SVR (από πριν εκπαιδευμένο)
print("\nΈναρξη Δημιουργίας Προβλέψεων:")
for i, date in enumerate(future_dates):
    # Κανονικοποιούμε την τελευταία γραμμή με βάση τα mean και std των δεδομένων εκπαίδευσης
    latest_features_scaled = (latest_features - X_train.mean()) / X_train.std()

    # Αν υπάρχουν NaN μετά την κανονικοποίηση, τα αντικαθιστούμε με 0
    if np.any(np.isnan(latest_features_scaled)):
        print(f"Εντοπίστηκε NaN κατά την κανονικοποίηση των δεδομένων για την πρόβλεψη {i + 1}. Αντικαθίστανται με 0.")
        latest_features_scaled = np.nan_to_num(latest_features_scaled)

    # Πρόβλεψη για μία ημέρα
    prediction = svr.predict([latest_features_scaled])[0]  # Εκτέλεση πρόβλεψης με το μοντέλο
    future_predictions.append(prediction)
    print(f"Πρόβλεψη για {date.date()}: {prediction}")

    # Ενημερώνουμε το τελευταίο feature με την πρόβλεψη για τη νέα τιμή
    latest_features["7- Day Standard Deviation of Daily Returns_lag_7"] = prediction

In [ ]:
# Δημιουργία DataFrame για μελλοντικές προβλέψεις
future_df = pd.DataFrame({
    "Date": future_dates,
    "Prediction": future_predictions
})

# Εκτύπωση DataFrame
print("\nΠροβλέψεις για τις Επόμενες 7 Ημέρες:")
print(future_df)

# <span style="color:yellow">Οπτικοποίηση Αποτελεσμάτων

In [ ]:
# Δημιουργία του διαγράμματος για το Training Set
plt.figure(figsize=(12, 6))

# Προβολή των πραγματικών τιμών (y_train) στο διάγραμμα
plt.plot(y_train.values, label="Πραγματικές Τιμές", color="blue", linestyle="-")

# Προβολή των προβλέψεων του μοντέλου (y_train_pred) στο διάγραμμα
plt.plot(y_train_pred, label="Προβλέψεις", color="green", linestyle="-")

# Προσθήκη τίτλου και ετικετών
plt.title("Σύγκριση Πραγματικών Τιμών και Προβλέψεων στο Training Set", fontsize=14)
plt.xlabel("Αριθμός Παρατήρησης", fontsize=12)
plt.ylabel("Τιμή", fontsize=12)

# Προσθήκη υπομνήματος
plt.legend()

# Εμφάνιση πλέγματος
plt.grid(alpha=0.4)

# Εμφάνιση του διαγράμματος
plt.show()

In [ ]:
# Δημιουργία του διαγράμματος
plt.figure(figsize=(12, 6))

# Προβολή των πραγματικών τιμών (y_test) στο διάγραμμα
plt.plot(y_test.values, label="Πραγματικές Τιμές", color="blue", linestyle="-")

# Προβολή των προβλέψεων του μοντέλου (y_pred) στο διάγραμμα
plt.plot(y_pred, label="Προβλέψεις", color="orange", linestyle="-")

# Προσθήκη τίτλου και ετικετών
plt.title("Σύγκριση Πραγματικών Τιμών και Προβλέψεων", fontsize=14)
plt.xlabel("Αριθμός Παρατήρησης", fontsize=12)
plt.ylabel("Τιμή", fontsize=12)

# Προσθήκη υπομνήματος
plt.legend()

# Εμφάνιση πλέγματος
plt.grid(alpha=0.4)

# Εμφάνιση του διαγράμματος
plt.show()

In [ ]:
# Υπολογισμός των σφαλμάτων πρόβλεψης (Residuals) για το Training Set
residuals_train = y_train.values - y_train_pred

# Διάγραμμα Διασποράς για τα Σφάλματα Πρόβλεψης στο Training Set
plt.figure(figsize=(12, 6))

# Scatter Plot των Σφαλμάτων
plt.scatter(range(len(residuals_train)), residuals_train, color="purple", alpha=0.6,
            label="Σφάλματα Πρόβλεψης (Residuals)")

# Χωριστή γραμμή για το μηδέν (Zero Line)
plt.axhline(y=0, color="red", linestyle="--", label="Μηδενική Γραμμή (Zero Line)")

# Προσθήκη τίτλου και ετικετών
plt.title("Διάγραμμα Σφαλμάτων Πρόβλεψης (Residuals) - Training Set", fontsize=14)
plt.xlabel("Αριθμός Παρατήρησης", fontsize=12)
plt.ylabel("Σφάλμα (Residual)", fontsize=12)

# Προσθήκη Υπομνήματος
plt.legend()

# Εμφάνιση πλέγματος
plt.grid(alpha=0.4)

# Εμφάνιση του Διαγράμματος
plt.show()

In [ ]:
# Υπολογισμός των σφαλμάτων πρόβλεψης (Residuals)
residuals = y_test.values - y_pred

# Διάγραμμα Διασποράς για τα Σφάλματα Πρόβλεψης
plt.figure(figsize=(12, 6))

# Scatter Plot των Σφάλμάτων
plt.scatter(range(len(residuals)), residuals, color="purple", alpha=0.6, label="Σφάλματα Πρόβλεψης (Residuals)")

# Χωριστή γραμμή για το μηδέν (Zero Line)
plt.axhline(y=0, color="red", linestyle="--", label="Μηδενική Γραμμή (Zero Line)")

# Προσθήκη τίτλου και ετικετών
plt.title("Διάγραμμα Σφαλμάτων Πρόβλεψης (Residuals) - SVR Model", fontsize=14)
plt.xlabel("Αριθμός Παρατήρησης", fontsize=12)
plt.ylabel("Σφάλμα (Residual)", fontsize=12)

# Προσθήκη Υπομνήματος
plt.legend()

# Εμφάνιση πλέγματος
plt.grid(alpha=0.4)

# Εμφάνιση του Διαγράμματος
plt.show()

In [ ]:
# Διάγραμμα Γραμμής για την Εξέλιξη των Σφαλμάτων στο Training Set
plt.figure(figsize=(12, 6))

# Line Plot για την Εξέλιξη των Σφαλμάτων στο Training Set
plt.plot(residuals_train, color="orange", label="Σφάλματα Πρόβλεψης (Residuals)", linewidth=2)

# Χωριστή γραμμή για το μηδέν (Zero Line)
plt.axhline(y=0, color="red", linestyle="--", label="Μηδενική Γραμμή (Zero Line)")

# Προσθήκη τίτλου και ετικετών
plt.title("Χρονική Εξέλιξη Σφαλμάτων Πρόβλεψης - Training Set", fontsize=14)
plt.xlabel("Αριθμός Παρατήρησης", fontsize=12)
plt.ylabel("Σφάλμα (Residual)", fontsize=12)

# Προσθήκη Υπομνήματος
plt.legend()

# Εμφάνιση Πλέγματος
plt.grid(alpha=0.4)

# Εμφάνιση του Διαγράμματος
plt.show()

In [ ]:
# Διάγραμμα Γραμμής για την Εξέλιξη των Σφαλμάτων
plt.figure(figsize=(12, 6))

# Line Plot για την Εξέλιξη των Σφαλμάτων
plt.plot(residuals, color="orange", label="Σφάλματα Πρόβλεψης (Residuals)", linewidth=2)

# Χωριστή γραμμή για το μηδέν (Zero Line)
plt.axhline(y=0, color="red", linestyle="--", label="Μηδενική Γραμμή (Zero Line)")

# Προσθήκη τίτλου και ετικετών
plt.title("Χρονική Εξέλιξη Σφαλμάτων Πρόβλεψης - SVR Model", fontsize=14)
plt.xlabel("Αριθμός Παρατήρησης", fontsize=12)
plt.ylabel("Σφάλμα (Residual)", fontsize=12)

# Προσθήκη Υπομνήματος
plt.legend()

# Εμφάνιση Πλέγματος
plt.grid(alpha=0.4)

# Εμφάνιση του Διαγράμματος
plt.show()

In [ ]:
# Φιλτράρισμα του test set, αν χρειάζεται
test_data_filtered = y_test.copy()  # Το y_test είναι το test set με τις πραγματικές τιμές

# Φιλτράρισμα δεδομένων από την ημερομηνία 2023-11-01 και μετά
test_data_filtered = test_data_filtered.loc["2023-11-01":]

# Προσθήκη των μελλοντικών προβλέψεων στο test_data_filtered ώστε να έχουν συνεχή καμπύλη
test_data_filtered = test_data_filtered.reindex(
    test_data_filtered.index.union(future_dates)
)  # Προσθήκη των μελλοντικών ημερομηνιών στο index
test_data_filtered.loc[future_dates] = future_predictions  # Προσθήκη των προβλέψεων στις νέες ημερομηνίες

# Δημιουργία διαγράμματος
plt.figure(figsize=(16, 8))

# Εμφάνιση ενιαίας μπλε γραμμής (Παρατηρήσεις και Προβλέψεις)
plt.plot(
    test_data_filtered.index,
    test_data_filtered.values,
    linestyle='-',
    color="blue",
    linewidth=2,
    label="Ενιαία Γραμμή Παρατηρήσεων και Προβλέψεων"
)

# Κίτρινα Bullets μόνο για τις Προβλέψεις
future_predictions_df = pd.DataFrame({
    "Date": future_dates,
    "Prediction": future_predictions
}).set_index("Date")
plt.scatter(
    future_predictions_df.index,
    future_predictions_df["Prediction"].values,
    color="yellow",
    edgecolor="black",
    label="Προβλέψεις (Bullets)",
    s=80,
    zorder=3
)

# Προσθήκη τίτλων, υπόμνημα και αξόνων
plt.title("Παρατηρήσεις και Συνέχεια με Προβλέψεις (Από 2023-11-01)", fontsize=18, fontweight='bold')
plt.xlabel("Ημερομηνία", fontsize=15)
plt.ylabel("Τιμή", fontsize=15)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=14, loc="upper left")

# Βελτιστοποίηση διάταξης
plt.tight_layout()

# Εμφάνιση διαγράμματος
plt.show()

In [ ]:
# Οπτικοποίηση προβλέψεων
plt.figure(figsize=(10, 6))
plt.plot(future_df["Date"], future_df["Prediction"], marker="o", label="Πρόβλεψη (Next 7 Days)", linestyle="-")
plt.title("Προβλέψεις για τις Επόμενες 7 Ημέρες", fontsize=16)
plt.xlabel("Ημερομηνία", fontsize=14)
plt.ylabel("Τιμή Πρόβλεψης", fontsize=14)
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.show()

In [ ]:
# Επιλέγουμε τα δεδομένα μόνο για τις τελευταίες 7 παρατηρήσεις
actual_last_7 = y_test.values[-7:]  # Οι πραγματικές τιμές για τις τελευταίες 7 ημέρες
predicted_last_7 = y_pred[-7:]  # Οι προβλέψεις για τις τελευταίες 7 ημέρες

# Δημιουργία διαγράμματος
plt.figure(figsize=(10, 6))

# Οπτικοποίηση των πραγματικών τιμών (τελευταίες 7 ημέρες)
plt.plot(range(1, 8), actual_last_7, label='Πραγματικές Τιμές', color='blue', linestyle='-', marker='o', linewidth=2)

# Οπτικοποίηση των προβλέψεων (τελευταίες 7 ημέρες)
plt.plot(range(1, 8), predicted_last_7, label='Προβλέψεις', color='red', linestyle='--', marker='x', linewidth=2)

# Προσθήκη τίτλου και ετικετών
plt.title('Σύγκριση Προβλέψεων με Πραγματικές Τιμές - Τελευταίες 7 Ημέρες', fontsize=14)
plt.xlabel('Ημέρα', fontsize=12)
plt.xticks(range(1, 8), labels=[f'Ημέρα {i}' for i in range(1, 8)])  # Ονόματα για τις τελευταίες 7 ημέρες
plt.ylabel('Μεταβλητότητα', fontsize=12)

# Προσθήκη πλέγματος
plt.grid(alpha=0.4)

# Προσθήκη υπομνήματος
plt.legend(fontsize=12)

# Εμφάνιση του διαγράμματος
plt.show()